This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [89]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [90]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [120]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
database_code='FSE'
dataset_code='AFX_X'
start_date='2017-1-1'
end_date='2017-12-31'
url = f'https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.json?api_key={API_KEY}&start_date={start_date}&end_date={end_date}'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")

In [121]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [122]:
# 2 - function to convert the returned JSON object into a Python dictionary.
def convert_data_to_dictionary(json_data):
    data_dict = {}

    if "dataset_data" in json_data and "data" in json_data["dataset_data"]:
        data = json_data["dataset_data"]["data"]
        column_names = json_data["dataset_data"]["column_names"]
        # Only need some columns to meet the task goals (left out 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover')
        value_indices = [column_names.index(column) for column in ["Open", "High", "Low", "Close", "Change", "Traded Volume"]]

        for entry in data:
            # create the Date key of the dictionary entry
            date = entry[0]
            # create the value of the dictionary entry
            entry_dict = {column_names[i]: entry[i] for i in value_indices}
            data_dict[date] = entry_dict

    return data_dict

In [123]:
# 3 - function to calculate what the highest and lowest opening prices were for the stock in this period.
def calculate_highest_lowest_opening_prices(data_dictionary):
    if not data_dictionary:
        return None, None

    opening_prices = [entry["Open"] for entry in data_dictionary.values() if "Open" in entry and entry["Open"] is not None]
    
    if opening_prices:
        highest_opening_price = max(opening_prices)
        lowest_opening_price = min(opening_prices)
        return highest_opening_price, lowest_opening_price
    else:
        return None, None


In [124]:
# 4 - function to calculate the largest change in any one day (based on High and Low price)
def calculate_largest_change(data_dictionary):
    if not data_dictionary:
        return None
    
    valid_changes = [entry["High"] - entry["Low"] for entry in data_dictionary.values() if "High" in entry and "Low" in entry and entry["High"] is not None and entry["Low"] is not None]
    
    if not valid_changes:
        return None
    
    largest_change = max(valid_changes)
    return round(largest_change, 2)

In [125]:
# 5 - function to calculate the largest change between any two days (based on Closing Price)
def calculate_largest_closing_change(data_dictionary):
    if not data_dictionary:
        return None
    
    closing_prices = [entry["Close"] for entry in data_dictionary.values() if "Close" in entry and entry["Close"] is not None]
    
    if len(closing_prices) <= 1:
        return None
    
    largest_change = max([abs(closing_prices[i] - closing_prices[i+1]) for i in range(len(closing_prices)-1)])
    return round(largest_change, 2)

In [126]:
# 6 - function to calculate the average daily trading volume
def calculate_average_trading_volume(data_dictionary):
    if not data_dictionary:
        return None
    
    trading_volumes = [entry["Traded Volume"] for entry in data_dictionary.values() if "Traded Volume" in entry and entry["Traded Volume"] is not None]

    if not trading_volumes:
        return None
    
    average_volume = sum(trading_volumes) / len(trading_volumes)
    return int(average_volume)

In [129]:
# 7 - function to calculate the median trade volume
def calculate_median_trading_volume(data_dictionary):
    trading_volumes = [entry["Traded Volume"] for entry in data_dictionary.values() if "Traded Volume" in entry and entry["Traded Volume"] is not None]
    trading_volumes.sort()
    num_volumes = len(trading_volumes)
    
    if num_volumes == 0:
        return None

    # if the total trade volumes is ODD then take the middle value
    if num_volumes % 2 == 1:
        median_volume = trading_volumes[num_volumes // 2]
    
    # if the total trade volumes is EVEN then take the average of the two middle values
    else:
        middle_indices = [num_volumes // 2 - 1, num_volumes // 2]
        median_volume = sum(trading_volumes[index] for index in middle_indices) / 2

    return int(median_volume)

In [130]:
data_dictionary = convert_data_to_dictionary(data)
# data_dictionary sample
# {'2017-12-29': {'Open': 51.76, 'High': 51.94, 'Low': 51.45, 'Close': 51.76, 'Change': None, 'Traded Volume': 34640.0},
#  '2017-12-28': {'Open': 51.65, 'High': 51.82, 'Low': 51.43, 'Close': 51.6, 'Change': None, 'Traded Volume': 40660.0}, 
# ...

# 3 - highest - lowest opening price
highest_opening, lowest_opening = calculate_highest_lowest_opening_prices(data_dictionary)
if highest_opening is not None and lowest_opening is not None:
    print("Highest Opening Price:", highest_opening)
    print("Lowest Opening Price:", lowest_opening)
else:
    print("No valid opening prices found.")

# 4 - largest change
largest_change = calculate_largest_change(data_dictionary)
if largest_change is not None:
    print("Largest Change in One Day:", largest_change)
else:
    print("No valid data found for calculating largest change.")
    
# 5 - largest closing change
largest_closing_change = calculate_largest_closing_change(data_dictionary)
if largest_closing_change is not None:
    print("Largest Change in Closing Price:", largest_closing_change)
else:
    print("No valid data found for calculating largest closing change.")

# 6 - average trading volume
average_trading_volume = calculate_average_trading_volume(data_dictionary)
if average_trading_volume is not None:
    print("Average Daily Trading Volume:", average_trading_volume)
else:
    print("No valid trading volume data found.")

# 7 - median
median_trading_volume = calculate_median_trading_volume(data_dictionary)
if median_trading_volume is not None:
    print("Median Trading Volume:", median_trading_volume)
else:
    print("No valid trading volume data found.")

Highest Opening Price: 53.11
Lowest Opening Price: 34.0
Largest Change in One Day: 2.81
Largest Change in Closing Price: 2.56
Average Daily Trading Volume: 89124
Median Trading Volume: 76286
